In [1]:
%run python_help.py

In [2]:
file_name = "./data/fl_100_1"
input_data = open(file_name).read()
facilities,customers = read_data(input_data)

customer_cost = gen_customer_cost(facilities,customers)
customer_damend = pd.Series(list(map(lambda x:x.demand,customers ))).astype("int32")
facility_cost = pd.Series(list(map(lambda x:x.setup_cost,facilities))).astype("int32")
facility_capacity = pd.Series(list(map(lambda x:x.capacity,facilities))).astype("int32")
facility_size = len(facilities)
customer_size = len(customers)
facility_min_size = gen_min_facility_size(facilities,customers)
facility_min_size

7

In [3]:
# model = Model("lr-facility")
# fs_vtype = "C"
# cs_vtype = "C"
# fs = facility_cost.map(lambda x: model.addVar(vtype=fs_vtype,lb=0,ub=1))
# cs = customer_cost.applymap(lambda x: model.addVar(vtype=cs_vtype,lb=0,ub=1))

In [4]:
# objective = quicksum(fs * facility_cost) + quicksum(cs.values.reshape(-1) * customer_cost.astype("int32").values.reshape(-1))

In [5]:
# for index,row in cs.iterrows():
#     model.addCons(quicksum(row) == 1)
#     model.addConsSOS1(row)

In [6]:
# for f,fc,c in zip(fs,facility_capacity,cs):
#     damend = quicksum(cs[c] * customer_damend)
#     model.addCons(damend <= f * fc )

In [7]:
# model.setObjective(objective)
# model.setRealParam("limits/time", 30)

In [8]:
# model.optimize()

In [3]:
def takeVar(l):return list(filter(lambda x:type is Variable,l))

class MyModel(ModelHelp):
    def __init__(self,model,fs_vtype,cs_vtype):
        
        fs = facility_cost.map(lambda x: model.addVar(vtype=fs_vtype,lb=0,ub=1))
        cs = customer_cost.applymap(lambda x: model.addVar(vtype=cs_vtype,lb=0,ub=1))
        
        objective = quicksum(fs * facility_cost) + quicksum(cs.values.reshape(-1) * customer_cost.astype("int32").values.reshape(-1))
#         objective = quicksum(cs.values.reshape(-1) * customer_cost.astype("int32").values.reshape(-1))
        for index,row in cs.iterrows():
            model.addCons(quicksum(row) == 1)
            
        for f,fc,c in zip(fs,facility_capacity,cs):
            damend = quicksum(cs[c] * customer_damend)
            model.addCons(damend <= f * fc )
            
        model.setObjective(objective)
        self.model = model 
        self.objective = objective
        self.fs = fs
        self.cs = cs
        self.facilities = facilities
        self.customers = customers

In [4]:
lr_model = MyModel(Model("lr-facility"),fs_vtype="C",cs_vtype="B")
lr_model.set_min_facility(facility_min_size)
lr_model.set_limit_time(300)
lr_model.optimize()

{'status': 'optimal',
 'time': 6.0,
 'total_time': 8.0,
 'obj ': 19328802.3075,
 'cost': 136385786.83011374}

In [5]:
fs_value = lr_model.fs_value()
print(lr_model.fs_one())
print(lr_model.fs_zero())

Int64Index([], dtype='int64')
Int64Index([ 0,  2,  3,  4,  6, 13, 14, 18, 25, 26, 38, 42, 43, 44, 45, 55, 65,
            71, 75, 77, 81, 86, 95, 97, 98],
           dtype='int64')


In [6]:
import gc
del lr_model
gc.collect()


400400

In [7]:
# for i in range(facility_min_size,min(facility_min_size + 10,len(facilities))):
#     mip_model = MyModel(Model("lr-facility"),fs_vtype="B",cs_vtype="B")
#     mip_model.set_facility_size(i)
#     print(i,mip_model.optimize())


In [ ]:
mip_model = MyModel(Model("mip-facility"),fs_vtype="B",cs_vtype="B")
# for value in mip_model.fs[fs_value <= 0.0 ]:
#     mip_model.model.fixVar(value,0)
# for value in mip_model.fs[fs_value >= 1 ]:
#     mip_model.model.fixVar(value,1)
mip_model.set_facility_size(facility_min_size+1)

# mip_model.set_min_facility(facility_min_size)
# mip_model.set_max_facility(facility_min_size + 0)
# mip_model.set_limit_time(30*60)
mip_model.optimize()

In [19]:
solution = mip_model.solution
used = mip_model.used
cost = mip_model.cost
print(mip_model.cost)

34513163.41140954


In [18]:
file = open("fl_500_7.result","w")


In [20]:
output_data = '%.2f' % cost + ' ' + str(0) + '\n'
output_data += ' '.join(map(str, solution))

In [21]:
file.write(output_data)
file.close()